In [42]:
# https://www.takeiho.com/python-blog-scrayping
# ブログの構造が変わって失敗した
# スクレイピングがしにくくなっていて
# かつもともとの目的がトップページのみだと一部のみしか達成できない
# for スクレイピング
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

# 出力フォーマットの定義
columns = ['title', 'date', 'url']
#   'category',は少なくとも変わった後のトップページには表示されてない
df = pd.DataFrame(columns=columns)

# スクレイピング先URL
# url = 'https://manablog.org/'
url = 'https://dividable.net/'

# 複数カテゴリ付いている場合に分割する
def fetchCategories(categories):
  categoriList = []
  for category in categories:
    categoriList.append(category.text)
  return ' '.join(categoriList)
  
# ブログ記事取得
def getTargetPageData(targetUrl):
  print(targetUrl)
  html = urlopen(targetUrl)
  soup = BeautifulSoup(html, 'html.parser')
  entries = soup.find_all('article')
  global df
  # print(entries)
  for entry in entries:
      # print(entry.find('h3').text) #title
      # print(entry.find('span', class_='post-content-time__created'))
      # findの検索は完全一致しか検索してくれない
      # 部分一致でほしかったら正規表現が必要だけど割とわからん
      # 今回はpost-content-time__の後にuploadかcreatedかでわざと分けて会ってそれで躓く
      
      se = pd.Series([
          entry.find('h3').text,  #title
          # fetchCategories(entry.find('p', class_='cat').find_all('a')), #category
          entry.find('span', class_='post-content-time__created'),  #date
          entry.find('h3').find('a').get('href') #url
      ], columns)
      df = df.append(se, columns)
      
  nextPage = soup.find('div', class_='pull-right')   
  # ページ構成が変わりpull-rightというcssセレクタがなかった
  # 昔の記事はアーカイブとして保存してあり次へがない
  if nextPage != None:
    nextPageUrl = nextPage.find('a')
    if nextPageUrl:
      getTargetPageData(nextPageUrl.get('href'))
  else:
    print('Done')

if __name__ == '__main__':
  getTargetPageData(url)
  
  # 取得したデータを表示
  print(df)


https://dividable.net/
Done
                                               title  ...                                                url
0                   転職活動中になぜつらくなるのか、3つの理由と対処法をまとめてみた  ...  https://dividable.net/nisotsu-tenshoku/tensyok...
1                     転職が怖いのはなぜ？3つの理由と解決方法についてまとめてみた  ...  https://dividable.net/nisotsu-tenshoku/tensyok...
2                          PythonでCSVを扱う方法について解説してみた  ...           https://dividable.net/python/python-csv/
3          新入社員だから転職できないと思っている人へ  新入社員は転職のゴールデンタイムです  ...  https://dividable.net/nisotsu-tenshoku/freshma...
4                             転職の真のリスクは、判断を先延ばしにすること  ...  https://dividable.net/nisotsu-tenshoku/job-cha...
5                              転職先が決まらない人がやっている4つのこと  ...  https://dividable.net/nisotsu-tenshoku/job-not...
6                          新卒で会社を辞めるべき？採用担当の視点で考えてみた  ...  https://dividable.net/nisotsu-tenshoku/freshma...
7  【保存版】Python BeautifulSoupの基礎と使い方~実際にデータを整形しつつダ...  ...  https://dividable.net/python/python-beaut